<a href="https://colab.research.google.com/github/DeependraChaddha/RAG_Projects/blob/main/RAG_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SET UP ENVIRONMENT

INSTALL PACKAGES

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube

SETTING UP LANGSMITH

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']= 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']=###YOUR API KEY###

SETTING OPENAI

In [ ]:
os.environ['OPENAI_API_KEY']=###YOUR API KEY###

##ColBERT

ColBERT uses tokenizes the documents and the query separately by using different or same models. Then, the score of each document is decided by the sum of the maximum similarity of each query embedding to any of the document embeddings

In [ ]:
!pip install -U ragatouille

In [ ]:
from ragatouille import RAGPretrainedModel
RAG=RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [ ]:
import requests

def get_wikipedia_page(title:str ###title: str -> Title of the Wiki page###
                       ):
  ###Retrieves full text context from Wikipedia page and returns it as raw string###

  #Wikipedia API endpoint
  URL="https://en.wikipedia.org/w/api.php"

  #Parameters of the API Endpoint
  params={"action":"query",
          "format":"json",
          "titles":title,
          "prop":"extracts",
          "explaintext":True,
          }
  #Custom use-agent header
  headers={"User-Agent":"RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

  response=requests.get(URL,params=params, headers=headers)
  data=response.json()

  #Extracting page content
  page=next(iter(data["query"]["pages"].values()))

  return page["extract"] if "extract" in page else None
full_document=get_wikipedia_page("Elon Musk")




In [ ]:
RAG.index(collection=[full_document],
          index_name="Musk-123",
          max_document_length=180,
          split_documents=True,
          )

In [ ]:
results=RAG.search(query="What company did Elon Musk make?",k=5)
results